In [1]:
import pandas as pd
import json
#Garbage Collector
import gc
import pickle

from google.colab import drive

drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
flightDataPath = 'gdrive/My Drive/FlightDelayPrediction/FlightData/'
weatherDataPath = 'gdrive/My Drive/FlightDelayPrediction/WeatherData/'
valid_airports = ('ATL','CLT','DEN','1DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO')
years=['2016','2017']
reqd_flight_columns=set(('FlightDate','Quarter','Year','Month','DayofMonth','DepTime','DepDelay15','CRSDepTime','Origin','Dest',\
                     'DepDelayMinutes','ArrTime','CRSArrTime','ArrDel15','ArrDelayMinutes'))
master_df = pd.DataFrame()

# Generating Weather DataFrame

the corresponding airport id like 'JFK' has been appended to the weather data for each airport for each month in order to make the join between flight data and weather data easier. The same weather dataframe has been **saved as a pickle file** inorder to save computation during future runs of the program

In [0]:
weather_df = pd.DataFrame()

f = None
jsonOb = None


#Checks to see if previously generated pickle file is present, else generate new weather dictionary
try:
  pkl_file = open(weatherDataPath+'WeatherData.pkl','rb')
  weather_df = pickle.load(pkl_file)
  pkl_file.close()

except FileNotFoundError:

  pkl_file = open(weatherDataPath+'WeatherData.pkl','wb')

  i=False
  print('Computing!')

  for airport in valid_airports:
    for year in years:
      for month in range(1,12+1):
        windspeedKmph=[]
        FeelsLikeF=[]
        winddir16Point=[]
        FeelsLikeC=[]
        DewPointC=[]
        windspeedMiles=[]
        DewPointF=[]
        HeatIndexF=[]
        cloudcover=[]
        HeatIndexC=[]
        precipMM=[]
        weatherIconUrl=[]
        WindGustMiles=[]
        pressure=[]
        WindGustKmph=[]
        weatherDesc=[]
        visibility=[]
        weatherCode=[]
        tempC=[]
        tempF=[]
        WindChillF=[]
        WindChillC=[]
        winddirDegree=[]
        humidity=[]
        time=[]

        f = open(weatherDataPath + airport + '/' + year + '-' + str(month)+'.json')
        jsonOb = json.load(f)
        temp_df = pd.DataFrame(jsonOb['data']['weather'])
        hourly_data = temp_df['hourly']
        temp_df = temp_df.drop(columns=['hourly','astronomy'])

        temp_df = pd.concat([temp_df]*24,ignore_index=True)
        index =temp_df.columns.get_loc('date')
        temp_df = pd.DataFrame(sorted(temp_df.values,key=lambda x: x[index].split('-')[::-1]),columns=temp_df.columns)
        
        for i in range(len(hourly_data)):
          for j in range(len(hourly_data[0])):
            windspeedKmph.append(hourly_data[i][j]['windspeedKmph'])
            cloudcover.append(hourly_data[i][j]['cloudcover'])
            precipMM.append(hourly_data[i][j]['precipMM'])
            pressure.append(hourly_data[i][j]['pressure'])
            WindGustKmph.append(hourly_data[i][j]['WindGustKmph'])
            visibility.append(hourly_data[i][j]['visibility'])
            tempF.append(hourly_data[i][j]['tempF'])
            WindChillF.append(hourly_data[i][j]['WindChillF'])
            winddirDegree.append(hourly_data[i][j]['winddirDegree'])
            humidity.append(hourly_data[i][j]['humidity'])
            time.append(hourly_data[i][j]['time'])
            weatherCode.append(hourly_data[i][j]['weatherCode'])
            DewPointF.append(hourly_data[i][j]['DewPointF'])

        compiled_hourly = pd.DataFrame()
        compiled_hourly['windspeedKmph']=windspeedKmph
        compiled_hourly['DewPointF']=DewPointF
        compiled_hourly['cloudcover']=cloudcover
        compiled_hourly['precipMM']=precipMM
        compiled_hourly['pressure']=pressure
        compiled_hourly['WindGustKmph']=WindGustKmph
        compiled_hourly['visibility']=visibility
        compiled_hourly['weatherCode']=weatherCode
        compiled_hourly['tempF']=tempF
        compiled_hourly['WindChillF']=WindChillF
        compiled_hourly['winddirDegree']=winddirDegree
        compiled_hourly['humidity']=humidity
        compiled_hourly['time']=time
        temp_df = pd.concat([temp_df,compiled_hourly],axis=1)
        airportList = [airport for i in range(temp_df.shape[0])]
        temp_df['Airport']=airportList
        
        if i:
          weather_df = pd.concat([weather_df,temp_df],ignore_index=True)
        else:
          weather_df = temp_df
        i = True
        f.close()

  weather_df.drop(columns=['mintempC','maxtempF','mintempF','maxtempC','sunHour','uvIndex'],inplace=True)
  pickle.dump(weather_df,pkl_file)
  pkl_file.close()
      
  


In [4]:
weather_df.shape

(263160, 16)

In [5]:
weather_df.loc[40:50]

,totalSnow_cm,date,windspeedKmph,DewPointF,cloudcover,precipMM,pressure,WindGustKmph,visibility,weatherCode,tempF,WindChillF,winddirDegree,humidity,time,Airport
40,0.0,2016-01-02,12,31,53,0.0,1024,15,10,113,46,41,302,58,1600,ATL
41,0.0,2016-01-02,11,32,63,0.0,1023,15,10,119,46,41,303,59,1700,ATL
42,0.0,2016-01-02,11,32,73,0.0,1023,15,10,119,46,42,304,60,1800,ATL
43,0.0,2016-01-02,10,33,64,0.0,1023,14,10,119,45,41,302,64,1900,ATL
44,0.0,2016-01-02,9,33,55,0.0,1023,14,10,116,44,40,301,68,2000,ATL
45,0.0,2016-01-02,9,34,46,0.0,1023,13,10,116,43,39,299,71,2100,ATL
46,0.0,2016-01-02,9,34,47,0.0,1023,14,10,116,42,38,297,73,2200,ATL
47,0.0,2016-01-02,9,34,49,0.0,1024,15,10,116,42,38,295,74,2300,ATL
48,0.0,2016-01-03,10,34,50,0.0,1024,15,10,116,41,37,293,75,0,ATL
49,0.0,2016-01-03,10,34,56,0.0,1023,16,10,116,41,37,297,76,100,ATL


# Joining Flight Data of All Flights with the weather data of the departure airport of the day

In [6]:
try:
  joined_pkl = open(flightDataPath + 'JoinedData.pkl','rb')
  print('Loaded from File!')
  joined_df = pickle.load(joined_pkl)
  joined_pkl.close()

except FileNotFoundError:    
  print('Generating File!')
  joined_df = pd.DataFrame()
  temp_df = pd.DataFrame()

  i=False

  for year in years:
    for months in range(1,12+1):
      nearest_hour=[]
      temp_df = pd.read_csv(flightDataPath + year +'/' + str(months) + '.csv',low_memory=False)
      

      temp_df.dropna(subset=['DepTime','FlightDate','Origin'],inplace=True)
      temp_df.drop(columns=list(set(temp_df.columns)-reqd_flight_columns),inplace=True)
      
      
      for i in range(len(temp_df)):
        time = temp_df.iloc[i]['DepTime']
        hr = time // 100
        mm = time % 100
        if mm >= 30:
          hr += 1
          if hr == 24:
            hr  = 23
        time = hr * 100
        if time == 0:
          nearest_hour.append('0')
        else:
          nearest_hour.append(str(int(time)))
      temp_df['NearestDepTime']=nearest_hour         
      temp_df = pd.merge(temp_df,weather_df,left_on=['FlightDate','Origin','NearestDepTime'],right_on=['date','Airport','time'],sort=False)
      if i:
        joined_df = pd.concat([joined_df,temp_df],ignore_index=True, sort=False)
      else:
        joined_df = temp_df
      i=True
  joined_df.drop(columns=['Airport'],inplace=True)
  joined_pkl = open(flightDataPath + 'JoinedData.pkl','wb')
  pickle.dump(joined_df,joined_pkl)
  joined_pkl.close()

Generating File!


In [7]:
joined_df.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'FlightDate', 'Origin',
       'Dest', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'CRSArrTime',
       'ArrTime', 'ArrDelayMinutes', 'ArrDel15', 'NearestDepTime',
       'totalSnow_cm', 'date', 'windspeedKmph', 'DewPointF', 'cloudcover',
       'precipMM', 'pressure', 'WindGustKmph', 'visibility', 'weatherCode',
       'tempF', 'WindChillF', 'winddirDegree', 'humidity', 'time'],
      dtype='object')

In [8]:
joined_df.shape

(1855994, 30)

In [9]:
joined_df['ArrTime']

0          1610.0
1          1237.0
2          1010.0
3          1138.0
4          1401.0
            ...  
1855989     134.0
1855990     144.0
1855991     759.0
1855992    1831.0
1855993    2007.0
Name: ArrTime, Length: 1855994, dtype: float64

In [10]:
joined_df['Origin'].unique()

array(['SEA', 'JFK', 'PHX', 'MCO', 'DEN', 'MIA', 'DFW', 'IAH', 'LAX',
       'LAS', 'SFO', 'ORD', 'ATL', 'CLT', 'EWR'], dtype=object)

In [11]:
joined_df

,Year,Quarter,Month,DayofMonth,FlightDate,Origin,Dest,CRSDepTime,DepTime,DepDelayMinutes,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,NearestDepTime,totalSnow_cm,date,windspeedKmph,DewPointF,cloudcover,precipMM,pressure,WindGustKmph,visibility,weatherCode,tempF,WindChillF,winddirDegree,humidity,time
0,2016.0,1.0,1.0,1.0,2016-01-01,SEA,JFK,745.0,741.0,0.0,1602.0,1610.0,8.0,0.0,800,0.0,2016-01-01,7,23,0,0.0,1030,8,10,113,34,29,108,65,800
1,2016.0,1.0,1.0,1.0,2016-01-01,SEA,PHX,820.0,821.0,1.0,1204.0,1237.0,33.0,1.0,800,0.0,2016-01-01,7,23,0,0.0,1030,8,10,113,34,29,108,65,800
2,2016.0,1.0,1.0,1.0,2016-01-01,SEA,SFO,724.0,759.0,35.0,946.0,1010.0,24.0,1.0,800,0.0,2016-01-01,7,23,0,0.0,1030,8,10,113,34,29,108,65,800
3,2016.0,1.0,1.0,1.0,2016-01-01,SEA,DEN,759.0,754.0,0.0,1143.0,1138.0,0.0,0.0,800,0.0,2016-01-01,7,23,0,0.0,1030,8,10,113,34,29,108,65,800
4,2016.0,1.0,1.0,1.0,2016-01-01,SEA,IAH,738.0,740.0,2.0,1351.0,1401.0,10.0,0.0,800,0.0,2016-01-01,7,23,0,0.0,1030,8,10,113,34,29,108,65,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855989,2017.0,4.0,12.0,17.0,2017-12-17,ORD,SFO,2322.0,2312.0,0.0,208.0,134.0,0.0,0.0,2300,0.0,2017-12-17,10,34,100,0.0,1016,12,6,122,37,34,219,90,2300
1855990,2017.0,4.0,12.0,17.0,2017-12-17,ORD,EWR,2120.0,2306.0,106.0,25.0,144.0,79.0,1.0,2300,0.0,2017-12-17,10,34,100,0.0,1016,12,6,122,37,34,219,90,2300
1855991,2017.0,4.0,12.0,17.0,2017-12-17,SEA,IAH,17.0,159.0,102.0,636.0,759.0,83.0,1.0,200,0.0,2017-12-17,10,38,100,0.8,1023,13,10,122,41,35,177,89,200
1855992,2017.0,4.0,12.0,17.0,2017-12-17,IAH,SFO,1635.0,1630.0,0.0,1858.0,1831.0,0.0,0.0,1700,0.0,2017-12-17,5,53,97,0.0,1019,6,10,122,60,60,113,78,1700
